## Análise da alavancagem do não-circulante

In [6]:
# bibliotecas usadas
import numpy as np
import pandas as pd

In [14]:
demonstracoes = pd.read_excel('D:\{estudos}\Python\ADC - Embraer S.A\Analise-Embraer\Demonstrações_Embraer_Ajustado.xlsm', sheet_name=None)
demonstracoes.keys()

<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Madu\AppData\Local\Temp\ipykernel_10296\71981632.py:1: SyntaxWarning: invalid escape sequence '\{'
  demonstracoes = pd.read_excel('D:\{estudos}\Python\ADC - Embraer S.A\Analise-Embraer\Demonstrações_Embraer_Ajustado.xlsm', sheet_name=None)


dict_keys(['Ativo', 'Passivo', 'DRE', 'DFC'])

In [15]:
# Extração dos dados da planilha
if demonstracoes:
    df_ativo = demonstracoes.get('Ativo')
    df_passivo = demonstracoes.get('Passivo')
    df_dre = demonstracoes.get('DRE')

    # verificada manualmente a correspondência dos códigos com os demonstrativos
    contas_chave = {
        "AtivoCirculante":      (df_ativo, "1.01"),
        "Estoque":              (df_ativo, "1.01.04"),
        "AtivoTotal":           (df_ativo, "1"),
        "PassivoCirculante":    (df_passivo, "2.01"),
        "PassivoNaoCirculante": (df_passivo, "2.02"),
        "PatrimonioLiquido":    (df_passivo, "2.03"),
        "ReceitaLiquida":       (df_dre, "3.01"),
        "LucroBruto":           (df_dre, "3.03"),
        "LucroOperacional":     (df_dre, "3.05"), # EBIT
        "LucroLiquido":         (df_dre, "3.11"),
        "LucroLiquidoSocios":   (df_dre, "3.11.01") # Lucro líquido atribuível aos sócios
    }
    
    # Extrai os dados para um DataFrame organizado por ano
    anos = [col for col in df_ativo.columns if isinstance(col, (int, float))]
    base_data = pd.DataFrame(index=contas_chave.keys(), columns=anos)

    print("\n--- Extraindo valores-chave dos demonstrativos ---")
    for nome, (df, codigo) in contas_chave.items():
        if df is not None:
            # A busca agora ocorre no dataframe com colunas já limpas
            linha = df[df['CD_CONTA'] == str(codigo)] # Garante que o código seja string
            if not linha.empty:
                valores = linha.iloc[0][anos]
                base_data.loc[nome] = valores
            else:
                print(f"   AVISO: Conta '{nome}' (código {codigo}) não encontrada. Será preenchida com 0.")
                base_data.loc[nome] = 0
    
    # Adiciona passivo total (soma)
    base_data.loc['PassivoTotal'] = base_data.loc['PassivoCirculante'] + base_data.loc['PassivoNaoCirculante']

    print("\n--- Base de Dados para Cálculo ---")
    print(base_data.to_string())


--- Extraindo valores-chave dos demonstrativos ---

--- Base de Dados para Cálculo ---
                                 2020             2021             2022             2023        2024
AtivoCirculante       40549746.680452  37967991.536853  32876200.424879     30670519.513    40326906
Estoque               16192719.426546  12832737.551855  13300908.386341     13351321.926    18181138
AtivoTotal            69691047.315529  65618880.512908  57920929.219613    54612924.7336    73219718
PassivoCirculante      16084627.91635  18279637.446943  18373669.443463    18565945.7574  27347016.0
PassivoNaoCirculante  34305057.823225  29409556.675324  23417345.436912    20652951.5502  25161794.0
PatrimonioLiquido     19301361.575954   17929686.39064  16129914.339238     15394027.426  20710908.0
ReceitaLiquida        22827248.501455  25011789.942734  24647341.804561  27419853.608272  35424174.0
LucroBruto             2797304.769881   3904740.967449   4950725.889129   4729220.957991   6382234.0
Luc

### Calculando os indicadores

In [21]:
indicadores = pd.DataFrame(columns=anos) # DataFrame para os resultados

# Liquidez
indicadores.loc['Liquidez Corrente'] = base_data.loc['AtivoCirculante'] / base_data.loc['PassivoCirculante']
indicadores.loc['Liquidez Seca'] = (base_data.loc['AtivoCirculante'] - base_data.loc['Estoque']) / base_data.loc['PassivoCirculante']
    
# Estrutura e Endividamento
indicadores.loc['Endividamento Geral (%)'] = (base_data.loc['PassivoTotal'] / base_data.loc['AtivoTotal']) * 100
indicadores.loc['Composição Endividamento (%)'] = (base_data.loc['PassivoCirculante'] / base_data.loc['PassivoTotal']) * 100
    
# Rentabilidade
indicadores.loc['Margem Bruta (%)'] = (base_data.loc['LucroBruto'] / base_data.loc['ReceitaLiquida']) * 100
indicadores.loc['Margem Operacional (%)'] = (base_data.loc['LucroOperacional'] / base_data.loc['ReceitaLiquida']) * 100
indicadores.loc['Margem Líquida (%)'] = (base_data.loc['LucroLiquido'] / base_data.loc['ReceitaLiquida']) * 100
indicadores.loc['ROA (%)'] = (base_data.loc['LucroOperacional'] / base_data.loc['AtivoTotal']) * 100
indicadores.loc['ROE (%)'] = (base_data.loc['LucroLiquido'] / base_data.loc['PatrimonioLiquido']) * 100

# Eficiência
indicadores.loc['Giro do Ativo'] = base_data.loc['ReceitaLiquida'] / base_data.loc['AtivoTotal']


print("\n\n--- Tabela Final de Indicadores Financeiros ---")
print(indicadores.round(2).to_string())



--- Tabela Final de Indicadores Financeiros ---
                                   2020       2021       2022       2023       2024
Liquidez Corrente              2.521025   2.077065   1.789311   1.651977   1.474636
Liquidez Seca                  1.514305   1.375041   1.065399   0.932848   0.809806
Endividamento Geral (%)       72.304389  72.676025  72.151838  71.812483  71.714029
Composição Endividamento (%)  31.920477  38.330774  43.965598  47.339285  52.080815
Margem Bruta (%)              12.254235  15.611601  20.086247  17.247433  18.016606
Margem Operacional (%)        -8.558666   4.652544  -2.369369   5.831401  10.648437
Margem Líquida (%)           -18.309114  -1.180509  -4.465961   3.003981   5.430842
ROA (%)                       -2.803384   1.773399  -1.008248   2.927808   5.151783
ROE (%)                      -21.653742  -1.646803  -6.824219   5.350693   9.288975
Giro do Ativo                  0.327549   0.381168   0.425534   0.502076   0.483806


### Exportar a tabela para excel

In [24]:
nome_arquivo_saida = "Indicadores_Financeiros_Embraer.xlsx"
try:
    # Cria um "escritor" de Excel para salvar múltiplas abas
    with pd.ExcelWriter(nome_arquivo_saida, engine='openpyxl') as writer:
        indicadores.to_excel(writer, sheet_name='Indicadores')
        base_data.to_excel(writer, sheet_name='Dados Base')
            
    print(f"\n\n Tabela de indicadores exportada com sucesso para o arquivo: '{nome_arquivo_saida}'")
    print("   O arquivo contém duas abas: 'Indicadores' e 'Dados Base'.")
except Exception as e:
    print(f"\n ERRO ao exportar para Excel: {e}")



 Tabela de indicadores exportada com sucesso para o arquivo: 'Indicadores_Financeiros_Embraer.xlsx'
   O arquivo contém duas abas: 'Indicadores' e 'Dados Base'.


### Principais indicadores para análise financeira
1. Indicadores de Liquidez

- Liquidez Corrente

    `Fórmula:` Ativo Circulante / Passivo Circulante

    Indica: A capacidade da empresa de pagar suas dívidas de curto prazo utilizando todos os seus ativos de curto prazo. Um valor maior que 1 é geralmente desejável.

- Liquidez Seca

    `Fórmula:` (Ativo Circulante - Estoques) / Passivo Circulante

    Indica: Similar à Liquidez Corrente, mas exclui os estoques, que são considerados menos líquidos. Mostra a capacidade de pagamento sem depender da venda de produtos.


2. Indicadores de Estrutura de Capital e Endividamento

- Participação de Capitais de Terceiros (Endividamento Geral)

    `Fórmula:` (Passivo Circulante + Passivo Não Circulante) / Ativo Total

    Indica: A proporção dos ativos da empresa que é financiada por capital de terceiros (dívidas). Quanto maior o indicador, maior o endividamento.

- Composição do Endividamento

    `Fórmula:` Passivo Circulante / (Passivo Circulante + Passivo Não Circulante)

    Indica: A porcentagem da dívida total que vence no curto prazo. Um valor menor sugere um perfil de dívida mais saudável e de longo prazo.


3. Indicadores de Rentabilidade

- Margem Bruta

    `Fórmula:` Lucro Bruto / Receita Líquida

    Indica: A rentabilidade das vendas após a dedução do custo dos produtos/serviços. Mostra a eficiência da produção e precificação.

- Margem Operacional (Margem EBIT)

    `Fórmula:` Lucro Operacional (EBIT) / Receita Líquida

    Indica: A rentabilidade gerada pelas operações principais da empresa, antes do resultado financeiro e dos impostos.

- Margem Líquida

    `Fórmula:` Lucro Líquido / Receita Líquida

    Indica: A porcentagem de cada R$ 1,00 de receita que efetivamente se transforma em lucro líquido após todas as despesas e impostos.

- Retorno sobre o Ativo (ROA)

    `Fórmula:` Lucro Operacional (EBIT) / Ativo Total

    Indica: A eficiência da empresa em gerar lucro a partir de todos os seus ativos. Mede a rentabilidade operacional do negócio.

- Retorno sobre o Patrimônio Líquido (ROE)

    `Fórmula:` Lucro Líquido / Patrimônio Líquido

    Indica: O retorno que os acionistas obtêm sobre o capital que investiram. É um dos principais indicadores de criação de valor para o investidor.


4. Indicadores de Eficiência

- Giro do Ativo

    `Fórmula:` Receita Líquida / Ativo Total

    Indica: Quantos reais de receita são gerados para cada R$ 1,00 investido em ativos. Quanto maior o giro, mais eficiente é a empresa na utilização de seus ativos.